In [1]:
import sys
sys.path.append('/host/d/Github/')
import glob
import os
import glob
import numpy as np
import nibabel as nb
import shutil
import pandas as pd
import whole_heart_segmentation_ZC.functions_collection as ff
import whole_heart_segmentation_ZC.Data_processing as Data_processing


/usr/local/lib/python3.8/dist-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
main_path ='/host/d/Data/WHS/current_challenge/'
save_path = '/host/d/Data/WHS/current_challenge/original_data'

In [26]:
files = ff.find_all_target_files(['*/center*/*.nii.gz'], main_path)

### re-organize the data

In [27]:
for f in files:
    center = os.path.basename(os.path.dirname(f))
    modality = os.path.basename(os.path.dirname(os.path.dirname(f)))
    
    basename = os.path.basename(f)
    if center == 'centerA' or center == 'centerC':
        patient_id = basename.split('_')[2]
    elif center == 'centerB' or center == 'centerE':
        patient_id = basename[4:8]
    
    # create a folder
    save_folder = os.path.join(save_path, modality, center, patient_id)
    ff.make_folder([os.path.join(save_path, modality), os.path.join(save_path, modality, center), save_folder])

    # find the corresponding image and label files
    img_file = ff.find_all_target_files(['*'+patient_id+'*_im*'],os.path.dirname(f))
    label_file = ff.find_all_target_files(['*'+patient_id+'*_la*'],os.path.dirname(f))

    # copy the file to the new folder
    img_file_des = os.path.join(save_folder,'image.nii.gz')
    label_file_des = os.path.join(save_folder,'label.nii.gz')
    if os.path.exists(img_file_des) is False:
        shutil.copyfile(img_file[0], img_file_des)
    if os.path.exists(label_file_des) is False:
        shutil.copyfile(label_file[0], label_file_des)
        

### print out the data info

In [ ]:
patient_list = ff.find_all_target_files(['*/center*/*'],os.path.join('/host/d/Data/WHS/current_challenge/original_data'))
print('Total number of patients: ', len(patient_list))

Total number of patients:  86


In [ ]:
results = []
for i in range(0,patient_list.shape[0]):
    patient_folder = patient_list[i]
    center = os.path.basename(os.path.dirname(patient_folder))
    modality = os.path.basename(os.path.dirname(os.path.dirname(patient_folder)))
    patient_id = os.path.basename(patient_folder)
    
    img_file = os.path.join(patient_folder,'image.nii.gz')
    label_file = os.path.join(patient_folder,'label.nii.gz')
    
    img_nii = nb.load(img_file)
    img_data = img_nii.get_fdata()

    
    img_shape = img_data.shape
    img_spacing = img_nii.header.get_zooms()
   
    
    results.append([  modality, center, patient_id, 
                      img_shape[0], img_shape[1], img_shape[2],
                      img_spacing[0], img_spacing[1], img_spacing[2]  ] )

results_df = pd.DataFrame(results, columns=['modality','center','patient_id',
                                            'size_x','size_y','size_z',
                                            'spacing_x','spacing_y','spacing_z'])
results_df.to_excel('/host/d/Data/WHS/Patient_lists/original_data_info.xlsx', index=False)

### resample to certain pixel spacing range

In [ ]:
patient_list = ff.find_all_target_files(['*/center*/*'],os.path.join('/host/d/Data/WHS/current_challenge/original_data'))
print('Total number of patients: ', len(patient_list))
basic_info = pd.read_excel('/host/d/Data/WHS/Patient_lists/data_info.xlsx',  dtype = {'patient_id': str})
for i in range(0,patient_list.shape[0]):
    patient_folder = patient_list[i]
    center = os.path.basename(os.path.dirname(patient_folder))
    modality = os.path.basename(os.path.dirname(os.path.dirname(patient_folder)))
    patient_id = os.path.basename(patient_folder)

    save_folder = os.path.join('/host/d/Data/WHS/current_challenge/processed_data', modality, center, patient_id)
    ff.make_folder([os.path.join('/host/d/Data/WHS/current_challenge/processed_data'), os.path.join('/host/d/Data/WHS/current_challenge/processed_data', modality), os.path.join('/host/d/Data/WHS/current_challenge/processed_data', modality, center), save_folder])

    row = basic_info[(basic_info['modality']==modality) & (basic_info['center']==center) & (basic_info['patient_id']==patient_id)]
    spacing = [row['spacing_x'].values[0], row['spacing_y'].values[0], row['spacing_z'].values[0]]

    if float(spacing[0])<0.8 or float(spacing[1])<0.8 or float(spacing[-1])<0.8:
        img_file = os.path.join(patient_folder,'image.nii.gz')
        img_nii = nb.load(img_file)
        label_file = os.path.join(patient_folder,'label.nii.gz')
        label_nii = nb.load(label_file)

        new_x = max(0.8, float(spacing[0]))
        new_y = max(0.8, float(spacing[1]))
        new_z = max(0.8, float(spacing[-1]))
        new_dim = [new_x, new_y, new_z]
        img_nii_resampled = Data_processing.resample_nifti(img_nii, order=0, mode='nearest', cval=np.min(img_nii.get_fdata()), in_plane_resolution_mm=new_dim[0], slice_thickness_mm=new_dim[-1])
        label_nii_resampled = Data_processing.resample_nifti(label_nii, order=0, mode='nearest', cval=0, in_plane_resolution_mm=new_dim[0], slice_thickness_mm=new_dim[-1])


        nb.save(img_nii_resampled, os.path.join(save_folder,'image.nii.gz'))
        nb.save(label_nii_resampled, os.path.join(save_folder,'label.nii.gz'))
    else:
        if not os.path.exists(os.path.join(save_folder,'image.nii.gz')):
            shutil.copyfile(os.path.join(patient_folder,'image.nii.gz'), os.path.join(save_folder,'image.nii.gz'))
        if not os.path.exists(os.path.join(save_folder,'label.nii.gz')):
            shutil.copyfile(os.path.join(patient_folder,'label.nii.gz'), os.path.join(save_folder,'label.nii.gz'))


      

Total number of patients:  86


In [ ]:
patient_list = ff.find_all_target_files(['*/center*/*'],os.path.join('/host/d/Data/WHS/current_challenge/processed_data'))
print('Total number of patients: ', len(patient_list))

results = []
for i in range(0,patient_list.shape[0]):
    patient_folder = patient_list[i]
    center = os.path.basename(os.path.dirname(patient_folder))
    modality = os.path.basename(os.path.dirname(os.path.dirname(patient_folder)))
    patient_id = os.path.basename(patient_folder)
    
    img_file = os.path.join(patient_folder,'image.nii.gz')
    label_file = os.path.join(patient_folder,'label.nii.gz')
    
    img_nii = nb.load(img_file)
    img_data = img_nii.get_fdata()

    
    img_shape = img_data.shape
    img_spacing = img_nii.header.get_zooms()
   
    
    results.append([  modality, center, patient_id, 
                      img_shape[0], img_shape[1], img_shape[2],
                      img_spacing[0], img_spacing[1], img_spacing[2]  ] )

results_df = pd.DataFrame(results, columns=['modality','center','patient_id',
                                            'size_x','size_y','size_z',
                                            'spacing_x','spacing_y','spacing_z'])
results_df.to_excel('/host/d/Data/WHS/Patient_lists/processed_data_info.xlsx', index=False)

Total number of patients:  86


### testing data (only image is provided, they are in the previous challenge)

##### re-organize

In [ ]:
main_path ='/host/d/Data/WHS/previous_challenge/'
save_path = '/host/d/Data/WHS/original_data'